In [26]:
import pickle
from datetime import datetime
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'],'size':8})
matplotlib.rcParams['pdf.fonttype'] = 42
from langdetect import detect, DetectorFactory


In [14]:
# load data
file_in = open("all_ads_labeled.pickle", "rb")
dat = pickle.load(file_in)

print("number of ads", dat.shape)

number of ads (602544, 17)


In [27]:
data = pd.read_csv("/Users/sofiepalmuskronborg/Desktop/Speciale/Data/ad_data.csv")

In [4]:
print("number of ads", data.shape)

number of ads (602544, 17)


In [33]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.max_colwidth")

In [8]:
pd.set_option("display.max_colwidth", None)  # No truncation for long text
data.head(5)

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,label,party,state
0,2020-02-12,"Every kid in America should have the same access to a high-quality public education—no matter where they live, the color of their skin, or how much money their parents make. Elizabeth has a comprehensive plan for our public education system. Her plan invests hundreds of billions of dollars in our public schools, paid for by a small wealth tax on fortunes above $50 million. It also funds schools equitably, invests in school infrastructure, fights back against segregation and discrimination, protects the rights of English Language Learners and immigrant students, better supports teachers, and provides a warm, safe, and nurturing school climate for all of our kids. \n\nIf you agree that we need to make big changes to improve public schools for our kids, sign up today. \n\n***\n\nTodos los estudiantes estadounidenses deberían tener el mismo acceso a una educación pública de alta calidad—sin importar dónde vivan, el color de su piel o cuánto dinero ganan sus padres. Elizabeth tiene un plan comprensivo para mejorar nuestro sistema de educación pública. Su plan invierte cientos de miles de millones de dólares, pagado por un pequeño impuesto sobre las grandes fortunas de más de $50 millones. También financia las escuelas públicas de manera adecuada, invierte en la infraestructura de los planteles escolares, lucha contra la segregación y discriminación, protege los derechos de los estudiantes aprendices de inglés y los estudiantes inmigrantes, apoya de mejor manera a los educadores y provee un ambiente escolar alegre, seguro y acogedor para todos los estudiantes. \n\nSi estás de acuerdo que necesitamos hacer grandes cambios para mejorar nuestras escuelas públicas para nuestros niños, inscríbete hoy.",49.5,499.5,"[{'percentage': '0.001175', 'region': 'Alabama'}, {'percentage': '0.003525', 'region': 'Kentucky'}, {'percentage': '0.031727', 'region': 'Arizona'}, {'percentage': '0.338425', 'region': 'California'}, {'percentage': '0.022327', 'region': 'Colorado'}, {'percentage': '0.049354', 'region': 'Florida'}, {'percentage': '0.010576', 'region': 'Georgia'}, {'percentage': '0.00235', 'region': 'Idaho'}, {'percentage': '0.06228', 'region': 'Illinois'}, {'percentage': '0.008226', 'region': 'Indiana'}, {'percentage': '0.007051', 'region': 'Michigan'}, {'percentage': '0.001175', 'region': 'Wyoming'}, {'percentage': '0.003525', 'region': 'Minnesota'}, {'percentage': '0.019976', 'region': 'New Jersey'}, {'percentage': '0.06698', 'region': 'New York'}, {'percentage': '0.023502', 'region': 'North Carolina'}, {'percentage': '0.003525', 'region': 'Oklahoma'}, {'percentage': '0.011751', 'region': 'Oregon'}, {'percentage': '0.025852', 'region': 'Pennsylvania'}, {'percentage': '0.164512', 'region': 'Texas'}, {'percentage': '0.001175', 'region': 'Delaware'}, {'percentage': '0.015276', 'region': 'Washington'}, {'percentage': '0.001175', 'region': 'Alaska'}, {'percentage': '0.001175', 'region': 'Missouri'}, {'percentage': '0.010576', 'region': 'Arkansas'}, {'percentage': '0.014101', 'region': 'Connecticut'}, {'percentage': '0.003525', 'region': 'Washington, District of Columbia'}, {'percentage': '0.012926', 'region': 'Iowa'}, {'percentage': '0.010576', 'region': 'Kansas'}, {'percentage': '0.00235', 'region': 'Louisiana'}, {'percentage': '0.001175', 'region': 'Maryland'}, {'percentage': '0.009401', 'region': 'Massachusetts'}, {'percentage': '0.007051', 'region': 'Nebraska'}, {'percentage': '0.0047', 'region': 'Virginia'}, {'percentage': '0.009401', 'region': 'Nevada'}, {'percentage': '0.003525', 'region': 'New Hampshire'}, {'percentage': '0.0047', 'region': 'New Mexico'}, {'percentage': '0.003525', 'region': 'Ohio'}, {'percentage': '0.003525', 'region': 'Rhode Island'}, {'percentage': '0.003525', 'region': 'Tennessee'}, {'percentage': '0.007051', 'region

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602544 entries, 0 to 602543
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ad_creation_time          602544 non-null  object 
 1   ad_creative_body          598057 non-null  object 
 2   spend                     602544 non-null  float64
 3   impressions               602544 non-null  float64
 4   delivery_by_region        463401 non-null  object 
 5   demographic_distribution  463401 non-null  object 
 6   page_id                   602544 non-null  int64  
 7   page_name                 602544 non-null  object 
 8   bylines                   600813 non-null  object 
 9   id                        602544 non-null  int64  
 10  spend_lo                  602544 non-null  int64  
 11  spend_hi                  602544 non-null  int64  
 12  impressions_lo            602544 non-null  int64  
 13  impressions_hi            602544 non-null  i

In [36]:
data.isnull().sum()

ad_creation_time                 0
ad_creative_body              4487
spend                            0
impressions                      0
delivery_by_region          139143
demographic_distribution    139143
page_id                          0
page_name                        0
bylines                       1731
id                               0
spend_lo                         0
spend_hi                         0
impressions_lo                   0
impressions_hi                   0
label                            0
party                            0
state                            0
dtype: int64

In [28]:
data.nunique()

ad_creation_time              1238
ad_creative_body             60635
spend                           42
impressions                     39
delivery_by_region          337533
demographic_distribution    462909
page_id                        520
page_name                      555
bylines                        957
id                          602544
spend_lo                        42
spend_hi                        42
impressions_lo                  39
impressions_hi                  39
label                            2
party                            3
state                           54
dtype: int64

## Cleaning the data

In [29]:
# Removing nans and empty strings (ad_creative_body, demographic_distribution, delivery_by_region)

# filter out nans in creative sentiment (e.g. they are probably videos), removes 4487 ads (0.74%)
data = data[~data['ad_creative_body'].isna()]
# filter out ads which just have empty space as text, e.g '  '(no 'real' text content), removes 40 in total out of (602544)
data = data[data['ad_creative_body'].str.strip() != '']
# filter out ads that do not have any targeting info (removes 138289 ads - 22.95% of all ads) - see analysis below
#data = data[~((data['demographic_distribution'].isna()) | (data['delivery_by_region'].isna()))]

# remove ads where demographic_distribution = [{'percentage': '1'}], not sure what that means, removes 6 ads
data = data[data['demographic_distribution'].map(lambda x: 1 if x == [{'percentage': '1'}] else 0) == 0]

# relabel Independents as Democrats - as the two independent politicians caucus with the Democrats
data['party'].replace('Independent','Democrat',inplace=True)

In [30]:
print("Number of ads after basic cleaning", data.shape)

Number of ads after basic cleaning (598017, 17)


In [31]:
data.to_csv('filtered_data.csv')

In [45]:
# remove all adds with low spend and low impressions

#dat = data[(data["spend_lo"] != 0) & (data["impressions_lo"] != 0)]

#print("Number of ads part 2", dat.shape)

Number of ads part 2 (139944, 17)


In [46]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 139944 entries, 67 to 602543
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ad_creation_time          139944 non-null  object 
 1   ad_creative_body          139944 non-null  object 
 2   spend                     139944 non-null  float64
 3   impressions               139944 non-null  float64
 4   delivery_by_region        139944 non-null  object 
 5   demographic_distribution  139944 non-null  object 
 6   page_id                   139944 non-null  int64  
 7   page_name                 139944 non-null  object 
 8   bylines                   139663 non-null  object 
 9   id                        139944 non-null  int64  
 10  spend_lo                  139944 non-null  int64  
 11  spend_hi                  139944 non-null  int64  
 12  impressions_lo            139944 non-null  int64  
 13  impressions_hi            139944 non-null  int64

In [ ]:
# Count the occurrences of each detected language
language_counts = data["language"].value_counts()
print(language_counts)

# Check how many ads are in English
english_ads = data[data["language"] == "en"]
print(f"Number of ads in English: {len(english_ads)}")

# Check if there are ads with multiple languages
# This might involve analyzing the text content for language patterns

In [5]:
sample_data.nunique()

ad_creation_time              1195
ad_creative_body             27767
spend                           37
impressions                     39
delivery_by_region           56203
demographic_distribution     77087
page_id                        496
page_name                      526
bylines                        873
id                          100000
spend_lo                        37
spend_hi                        37
impressions_lo                  39
impressions_hi                  39
label                            2
party                            3
state                           52
detected_languages              23
dtype: int64

In [23]:
### Prøv at splitte text

df_sample = data.tail(100)

In [25]:
import pandas as pd
from langdetect import detect
import re


# Assuming your DataFrame is named 'df' and the text column is 'ad_creative_body'
def detect_languages_in_ad(ad_text):
    # Split the text into paragraphs or sentences based on double newlines
    paragraphs = re.split(r"\n\n", ad_text)

    # Detect language for each paragraph
    detected_languages = []
    for idx, paragraph in enumerate(paragraphs):
        try:
            detected_lang = detect(paragraph)
            detected_languages.append((idx + 1, detected_lang))
        except:
            detected_languages.append((idx + 1, "unknown"))  # Handle detection errors

    return detected_languages


# Apply language detection to each row in the 'ad_creative_body' column
df_sample["detected_languages"] = df_sample["ad_creative_body"].apply(
    detect_languages_in_ad
)

# Print the updated DataFrame with detected languages
print(df_sample[["detected_languages"]])

        detected_languages
602444           [(1, en)]
602445           [(1, en)]
602446           [(1, en)]
602447           [(1, en)]
602448           [(1, en)]
...                    ...
602539           [(1, en)]
602540           [(1, en)]
602541           [(1, en)]
602542           [(1, en)]
602543  [(1, en), (2, en)]

[100 rows x 1 columns]


/var/folders/cr/2b4bjd0d6cscnr8h_66vmrqm0000gn/T/ipykernel_21385/341831627.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["detected_languages"] = df_sample["ad_creative_body"].apply(
